# Cahier des charges

| Fonction | Cartographier la distance au vélo disponible le plus proche en temps réel |
| --- | --- |
| Objectif | Réaliser une couche raster de la distance au vélo disponible le plus proche à partir des données de l'API. |
| Contraintes | - Utiliser le package *rasterio*<br>- La couche doit couvrir toute zone située à moins de 5km d'une station Velo'V<br>- La résolution spatiale doit être de 50m |

Pour vous aider, un *template raster* de la bonne taille et à la bonne résolution, initialisé avec des zéros, est disponible dans ```datasets/template_raster.tif```

## Résultat attendu
![Raster de distance attendu](img/distance.png "Raster de distance attendu")

# Algorithmie

Voici quelques grandes étapes pour vous aider à réaliser cet objectif. Ces étapes ne sont pas obligatoires, vous pouvez décider de procéder complètement autrement si vous le souhaitez. 

> ⚠️ La méthode présentée ici n'est pas optimale en production, mais a été choisie pour sa simplicité algorithmique

## 1. Récupérer les stations avec au moins un vélo disponible
On ouvre le tableau des stations et on filtre les stations avec au moins un vélo disponible.

## 2. Créer un raster aux bonnes dimensions et résolutions
On utilise le template_raster comme modèle pour en créer un nouveau.

## 3. Réaliser un masque des zones à plus de 5km d'une station
On réalise un buffer de 5km autour de chaque station (y compris celle qui n'ont pas de vélo disponible), puis on transforme ces buffers en un masque.

## 4. Calculer chaque pixel
Pour cette étape, on va réaliser une itération double : sur chaque ligne du raster, puis sur chaque colonne. 
Voici un exemple d'algorithme :

![Algorithme itération sur chaque pixel](img/Algo_CartoVelosProches.png "Algorithme itération sur chaque pixel")

Ci-dessous la description de ce qui est fait **pour chaque pixel du raster**.

### 4.1. Chercher la station la plus proche
On cherche la station avec au moins un vélo disponible la plus proche.

### 4.2. Mesurer la distance à cette station
On mesure la distance entre le point central du pixel et la station trouvée en 4.1.

### 4.3. Enregistrer cette distance dans la valeur du pixel
On stocke cette valeur dans le raster, à la position du pixel traité.

## 5. Ecrire le raster en sortie sur le disque
Une fois l'itération sur les pixels terminée, on peux enregistrer le raster sur le disque dur.

# Programmation

> ⚠️ La méthode présentée ici n'est pas optimale en production, mais a été choisie pour sa simplicité algorithmique

## 1. Récupérer les stations avec au moins un vélo disponible
Comme pour l'étape précédente, on cherche à charger dans une variable python le tableau des stations. *Geopandas* peut faire cela. 
Idem pour le filtrage, on le fait avec *pandas*.

Pour pouvoir utiliser la fonction nearest_points de shapely, nous aurons besoin de créer une unique géométrie de type MultiPoint contenant tous les points des stations.

In [ ]:
stations_points = MultiPoint(list(filtered_gdf.geometry))

## 2. Créer un raster aux bonnes dimensions et résolutions
On va utiliser rasterio pour créer le raster. Il est possible de le faire entièrement en Python, mais dans premier temps, nous allons utiliser un raster "template". 

In [3]:
import rasterio as rio

# Charger les métadonnées du raster
src = rio.open('datasets/template_raster.tif')

# Afficher le profil du raster
profile = src.profile
print(profile)

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': 255.0, 'width': 652, 'height': 771, 'count': 1, 'crs': CRS.from_epsg(3857), 'transform': Affine(50.0, 0.0, 526943.6377,
       0.0, -50.0, 5765599.8051), 'blockysize': 3, 'tiled': False, 'interleave': 'band'}


In [2]:
# Charger les données du raster
data = src.read(1)
print(data)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Il faut bien penser à stocker le profil du raster pour pouvoir en enregistrer un nouveau identique à la fin de notre programme. Une fois les données chargées en mémoire vive (assignées à une variable python), on peut fermer le template_raster :

In [4]:
src.close()

## 3. Réaliser un masque des zones à plus de 5km d'une station
On réalise un buffer de 5km autour de chaque station (y compris celle qui n'ont pas de vélo disponible), puis on transforme ces buffers en un masque.

Voici comment faire avec rasterio.

In [ ]:
from rasterio.features import geometry_mask

# On fait un buffer de 5km autour des stations pour réaliser le masque
buffers = gdf_stations.buffer(5000)

# On réalise le masque (on fait appel aux métadonnées du template raster pour que le masque soit à la bonne taille)
mask = geometry_mask(list(buffers), src.shape, src.transform)

## 4. Calculer chaque pixel
On va faire itérer deux variables (```row``` et ```col```) pour qu'elle prennent successivement tous les numéros de ligne et tous les numéros de colonne.

In [ ]:
# On itère sur chaque ligne du raster
for row in range(src.shape[0]):
    # On itère sur chaque colonne du raster
    for col in range(src.shape[1]):
        # Pour vérifier si le pixel est masqué ou non
        if mask[row, col]:
            # Pour modifier la valeur du pixel actuel sur le raster
            data[row, col] = valeur

### 4.1. Chercher la station la plus proche
Pour chercher la station la plus proche du pixel actuel, il faut d'abord créer un Point *shapely* au centre du pixel

In [ ]:
current_point = Point(src.xy(row, col))

On peux ensuite chercher le point le plus proche avec ```shapely.ops.nearest_points```. Cette fonction renvoie une paire de points stockée dans un tuple.

### 4.2. Mesurer la distance à cette station
Pour mesurer la distance entre le point central du pixel et la station trouvée en 4.1, on peux utiliser une méthode des objets ```shapely.Point```. 

### 4.3. Enregistrer cette distance dans la valeur du pixel
Voir l'exemple ci-dessus (en partie 4.) pour voir comment stocker la distance dans le raster.

## 5. Ecrire le raster en sortie sur le disque
On ouvre le raster en sortie en mode "write" (w), et on fait appel au profil enregistré précédemment pour que le raster soit aux bonnes dimensions, correctement projeté et à la bonne résolution.

La méthode dst.write prends comme arguments le tableau de données à écrire et le numéro de la bande sur laquelle écrire.

In [ ]:
with rio.open('datasets/output/distances.tif', 'w', **profile) as dst:
    dst.write(data, 1)